In [1]:
import h5py
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import svm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import geopandas as gpd
import pandas as pd

import numpy as np

np.random.seed(3108)

In [2]:
#getwd
import os
cwd = os.getcwd()
print(cwd)

C:\Users\matte\OneDrive\Documents\DSEO_project


In [8]:
# Directory of GeoJSON files
data_dir = "S3_June_11x11_allBands_cloud20_sampleregions"

# Find all .geojson files in the directory
geojson_files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith(".geojson")]

# Create a list of GeoDataFrames for each file without adding extra columns
gdfs = []
for file in geojson_files:
    gdf = gpd.read_file(file)  # Read each GeoJSON file
    gdfs.append(gdf)

# Concatenate all GeoDataFrames into one
full_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

# Display the first few rows of the GeoDataFrame
print(full_gdf.head())

# Display the final dimensions of the GeoDataFrame
print(f"Final dimensions of the GeoDataFrame: {full_gdf.shape}")

                       id                                              B11_3  \
0  0000000000000000050b_0  [ [ 0.23389999568462372, 0.23389999568462372, ...   
1  0000000000000000050c_0  [ [ 0.34615001082420349, 0.34615001082420349, ...   
2  0000000000000000050d_0  [ [ 0.22210000455379486, 0.22640000283718109, ...   
3  0000000000000000050e_0  [ [ 0.22775000333786011, 0.23170000314712524, ...   
4  0000000000000000050f_0  [ [ 0.25940001010894775, 0.25290000438690186, ...   

                                               B12_3  \
0  [ [ 0.16249999403953552, 0.16249999403953552, ...   
1  [ [ 0.2592499852180481, 0.2592499852180481, 0....   
2  [ [ 0.15710000693798065, 0.16134999692440033, ...   
3  [ [ 0.15884999930858612, 0.15729999542236328, ...   
4  [ [ 0.17700000107288361, 0.17509999871253967, ...   

                                                B2_3  \
0  [ [ 0.13369999825954437, 0.13369999825954437, ...   
1  [ [ 0.14524999260902405, 0.14524999260902405, ...   
2  [ [ 0.12330

In [9]:
# Save the concatenated GeoDataFrame as a GeoJSON file
full_gdf.to_file('data/merged_data_June_allbands_cloud20.geojson', driver='GeoJSON')

In [10]:
merged_data_april = gpd.read_file("data/merged_data_April_allbands_cloud20.geojson")
merged_data_june = gpd.read_file("data/merged_data_June_allbands_cloud20.geojson")
merged_data_october = gpd.read_file("data/merged_data_October_allbands_cloud20.geojson")

In [29]:
merged_data_april = merged_data_april.rename(columns={'B2_1': 'B2',  'B3_1': 'B3', 'B4_1': 'B4', 'B5_1': 'B5', 'B6_1': 'B6', 'B7_1': 'B7', 'B8_1': 'B8', 'B8A_1': 'B8A', 'B11_1': 'B11', 'B12_1': 'B12', 'NDVI_1': 'NDVI', 'EVI_1': 'EVI', 'EVI2_1': 'EVI2', 'SAVI_1': 'SAVI', 'NDWI_1': 'NDWI'})
merged_data_june = merged_data_june.rename(columns={'B2_3': 'B2',  'B3_3': 'B3', 'B4_3': 'B4', 'B5_3': 'B5', 'B6_3': 'B6', 'B7_3': 'B7', 'B8_3': 'B8', 'B8A_3': 'B8A', 'B11_3': 'B11', 'B12_3': 'B12', 'NDVI_3': 'NDVI', 'EVI_3': 'EVI', 'EVI2_3': 'EVI2', 'SAVI_3': 'SAVI', 'NDWI_3': 'NDWI'})
merged_data_october = merged_data_october.rename(columns={'B2_7': 'B2',  'B3_7': 'B3', 'B4_7': 'B4', 'B5_7': 'B5', 'B6_7': 'B6', 'B7_7': 'B7', 'B8_7': 'B8', 'B8A_7': 'B8A', 'B11_7': 'B11', 'B12_7': 'B12', 'NDVI_7': 'NDVI', 'EVI_7': 'EVI', 'EVI2_7': 'EVI2', 'SAVI_7': 'SAVI', 'NDWI_7': 'NDWI'})

In [30]:
full_gdf = pd.concat([merged_data_april, merged_data_june, merged_data_october], axis=0)

In [31]:
print(full_gdf.head())

# Display the final dimensions of the GeoDataFrame
print(f"Final dimensions of the GeoDataFrame: {full_gdf.shape}")
print(merged_data_october.shape)

                       id                                                B11  \
0  0000000000000000050c_0  [ [ 0.30860000848770142, 0.30860000848770142, ...   
1  0000000000000000050d_0  [ [ 0.18799999356269836, 0.19380000233650208, ...   
2  0000000000000000050e_0  [ [ 0.20035000145435333, 0.20675000548362732, ...   
3  0000000000000000050f_0  [ [ 0.2955000102519989, 0.27259999513626099, 0...   
4  00000000000000000511_0  [ [ 0.26890000700950623, 0.26890000700950623, ...   

                                                 B12  \
0  [ [ 0.2468000054359436, 0.2468000054359436, 0....   
1  [ [ 0.14790000021457672, 0.14990000426769257, ...   
2  [ [ 0.15270000696182251, 0.15710000693798065, ...   
3  [ [ 0.22560000419616699, 0.20800000429153442, ...   
4  [ [ 0.2062000036239624, 0.2062000036239624, 0....   

                                                  B2  \
0  [ [ 0.13500000536441803, 0.13500000536441803, ...   
1  [ [ 0.11129999905824661, 0.11050000041723251, ...   
2  [ [ 0.11820

In [ ]:
full_gdf.to_file('data/merged_data_allmonths_allbands_cloud20.geojson', driver='GeoJSON')

In [ ]:
# Print the first row of the GeoDataFrame
print(full_gdf.loc[1, 'B11_1'])

In [ ]:
# Group by 'l3_species' and count the number of rows for each species
species_counts = full_gdf['l3_species'].value_counts()

# Create a bar plot
plt.figure(figsize=(10, 6))
species_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Rows per Species')
plt.xlabel('Species')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()  # Adjust layout to prevent label clipping
plt.show()

In [ ]:
band_data = full_gdf.iloc[:, 1:-4]
print(band_data.head())

In [ ]:
import ast

# Apply ast.literal_eval to all columns that contain the band data
for band in band_data.columns:
    band_data[band] = band_data[band].apply(ast.literal_eval)

In [ ]:
import numpy as np

# Initialize the 4D array to store the reshaped data
rows = len(band_data)
num_bands = len(band_data.columns)  # Number of bands (21 in your case)
X = np.zeros((rows, 11, 11, num_bands), dtype=np.float32)

# Loop through the rows and fill the 4D array with the band data
for i in range(rows):
    # Get the list of all band columns for this row (converted to np arrays)
    stacked_bands = np.stack([
        np.array(band_data[band][i], dtype=np.float32) 
        for band in band_data.columns
    ], axis=-1)  # Stack along the last axis to create (11, 11, 21)

    # Assign the stacked 3D array to the corresponding slice in the 4D array
    X[i] = stacked_bands

In [ ]:
print(X.shape)

In [ ]:
# Save the concatenated GeoDataFrame as a GeoJSON file
full_gdf.to_file('data/merged_data_AprilJuneOctober_B2B3B4B6B8B11NDVI_cloud20.geojson', driver='GeoJSON')

In [ ]:
#create label file
# Assuming your data is loaded into df
# Get unique species
unique_species = full_gdf['l3_species'].unique()

# Create a new label DataFrame with rows = images and columns = species, initialized to 0
labels = pd.DataFrame(0, index=merged_data['id'], columns=unique_species)

# Populate the label DataFrame
for idx, row in merged_data.iterrows():
    species = row['l3_species']
    labels.loc[row['id'], species] = 1

# Reset the index for better readability (optional)
labels.reset_index(inplace=True)
labels.rename(columns={'index': 'id'}, inplace=True)
labels = labels.iloc[:, 1:]

# Show the resulting DataFrame
print(labels)

In [ ]:
labels = labels.values

print(labels)